In [ ]:
import os
import zipfile
import random
import shutil
import numpy as np 
import pandas as pd 
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
!pip install celluloid 
from celluloid import Camera


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if 'zip' in os.path.join(dirname, filename):
            zip_ref  = zipfile.ZipFile(os.path.join(dirname, filename), 'r')
            zip_ref.extractall('/kaggle/working')
        zip_ref.close()

        
source_path_train = '/kaggle/working/train'
source_path_test = '/kaggle/working/test1'

# creating dogs and cats directories
os.makedirs('/kaggle/working/images/dogs')
os.makedirs('/kaggle/working/images/cats')

os.makedirs('/kaggle/working/cats-v-dogs')

images_dogs = '/kaggle/working/images/dogs'
images_cats = '/kaggle/working/images/cats'

root_dir = '/kaggle/working/cats-v-dogs'

data_train_list = os.listdir(source_path_train)
cat_jpg_train = [jpg for jpg in data_train_list if 'cat' in jpg]
dog_jpg_train = [jpg for jpg in data_train_list if 'dog' in jpg]
# ls -R | grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//--/g' -e 's/^/   /' -e 's/-/|/'

In [ ]:
def move_jpg(photo_names: list, source_path: str, dest_path: str, split_data=0):
    
    # moving photos to subdirectories cats and dogs
    for photo in photo_names:
        shutil.move(os.path.join(source_path, photo), os.path.join(dest_path, photo))

# moving photos of cats to new directory 'trainig/cats'
move_jpg(cat_jpg_train, source_path_train, images_cats )

# moving photos of dogs to new subdirectory 'training/dogs'
move_jpg(dog_jpg_train, source_path_train, images_dogs )

In [ ]:
print(f'Number of images in train directory: {len(os.listdir(source_path_train))}')
print(f'Number of images in test directory: {len(os.listdir(source_path_test))}')
print(f'Number of images in dogs directory: {len(os.listdir(images_dogs))}')
print(f'Number of images in cats directory: {len(os.listdir(images_cats))}')

In [ ]:
if os.path.exists(root_dir):
    shutil.rmtree(root_dir)


def create_train_test_dirs(root_path):
    training_cats_dir = 'training/cats'
    training_dogs_dir = 'training/dogs'

    testing_cats_dir = 'testing/cats'
    testing_dogs_dir = 'testing/dogs'

    directories = [training_cats_dir, 
                 training_dogs_dir, 
                 testing_cats_dir, 
                 testing_dogs_dir]

    for dir in directories:
        os.makedirs(os.path.join(root_path, dir))

try:
    create_train_test_dirs(root_path=root_dir)
except FileExistsError:
    print("You should not be seeing this since the upper directory is removed beforehand")

In [ ]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    source_data_list = os.listdir(SOURCE)

    # testing for photos with size 0
    for photo in source_data_list:
        if os.path.getsize(os.path.join(SOURCE, photo)) == 0:
            source_data_list.remove(photo)
      
    # shuffled list of photos
    source_data_list = random.sample(source_data_list, len(source_data_list))

    # training 
    trainig_photos = source_data_list[ :int(len(source_data_list) * SPLIT_SIZE)]

    # test 
    testing_photos = source_data_list[int(len(source_data_list) * SPLIT_SIZE): ]

    # copying trainig data 
    for train_photo in trainig_photos:
        shutil.move( os.path.join(SOURCE, train_photo), os.path.join(TRAINING, train_photo) )

    # copying test data
    for test_photo in testing_photos:
        shutil.move( os.path.join(SOURCE, test_photo), os.path.join(TESTING, test_photo) )

In [ ]:
# Define paths
CAT_SOURCE_DIR = '/kaggle/working/images/dogs'
DOG_SOURCE_DIR = '/kaggle/working/images/cats'

TRAINING_DIR = "/kaggle/working/cats-v-dogs/training/"
TESTING_DIR = "/kaggle/working/cats-v-dogs/testing/"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
TESTING_CATS_DIR = os.path.join(TESTING_DIR, "cats/")

TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
TESTING_DOGS_DIR = os.path.join(TESTING_DIR, "dogs/")

if len(os.listdir(TRAINING_CATS_DIR)) > 0:
    for file in os.scandir(TRAINING_CATS_DIR):
        os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
    for file in os.scandir(TRAINING_DOGS_DIR):
        os.remove(file.path)
if len(os.listdir(TESTING_CATS_DIR)) > 0:
    for file in os.scandir(TESTING_CATS_DIR):
        os.remove(file.path)
if len(os.listdir(TESTING_DOGS_DIR)) > 0:
    for file in os.scandir(TESTING_DOGS_DIR):
        os.remove(file.path)


split_size = .9

split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

print(f"\n\nThere are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(TESTING_CATS_DIR))} images of cats for testing")
print(f"There are {len(os.listdir(TESTING_DOGS_DIR))} images of dogs for testing")

In [ ]:
# callback class
class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') >= 0.99):
            print('\nReached desired training accuracy of 99%. Training stopped!')
            self.model.stop_training = True

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

def create_model():
    
    model = tf.keras.models.Sequential([
                                      
              tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(150,150,3)), 
              tf.keras.layers.MaxPool2D(2,2),

              tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
              tf.keras.layers.MaxPool2D(2,2),

              tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
              tf.keras.layers.MaxPool2D(2,2),

              tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
              tf.keras.layers.MaxPool2D(2,2),

              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='relu'),
              tf.keras.layers.Dense(1, activation='sigmoid')  
        ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='binary_crossentropy',
                  metrics=['accuracy']) 

    return model


In [ ]:
def train_val_generators_aug(TRAINING_DIR, VALIDATION_DIR):
    
    train_datagen_aug = ImageDataGenerator( rescale = 1.0 / 255.,
                                        rotation_range=45,
                                        width_shift_range=0.25,
                                        height_shift_range=0.25,
                                        shear_range=0.25,
                                        zoom_range=0.25,
                                        horizontal_flip=True,
                                        fill_mode='nearest')


    train_generator_aug = train_datagen_aug.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=20,
                                                        class_mode='binary',
                                                        target_size=(150, 150))


    validation_datagen_aug = ImageDataGenerator( rescale = 1.0 / 255.)


    validation_generator_aug = validation_datagen_aug.flow_from_directory(directory=VALIDATION_DIR,
                                                                  batch_size=20,
                                                                  class_mode='binary',
                                                                  target_size=(150, 150))

    return train_generator_aug, validation_generator_aug

In [ ]:
train_generator_aug, validation_generator_aug = train_val_generators_aug(TRAINING_DIR, TESTING_DIR)
model_augumented = create_model()
callbacks = myCallbacks()
history = model_augumented.fit(train_generator_aug,
                    epochs=150,
                    batch_size=16, # this time batch size was added ver.3 previous stopped on 93% and 89%
                    verbose=1,
                    validation_data=validation_generator_aug,
                    callbacks=[callbacks])

In [ ]:
acc_aug=history.history['accuracy']
val_acc_aug=history.history['val_accuracy']
loss_aug=history.history['loss']
val_loss_aug=history.history['val_loss']

epochs_aug=range(len(acc_aug)) # Get number of epochs

plt.plot(epochs_aug, acc_aug, 'r', "Training Accuracy")
plt.plot(epochs_aug, val_acc_aug, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

plt.plot(epochs_aug, loss_aug, 'r', "Training Loss")
plt.plot(epochs_aug, val_loss_aug, 'b', "Validation Loss")
plt.show()